## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-02-21-17 +0000,nypost,Stefanik rips Hochul for endorsing ‘raging ant...,https://nypost.com/2025/11/09/us-news/stefanik...
1,2025-11-10-02-16-03 +0000,48hills,"With a ‘heavy heart,’ tenant protections move ...",https://48hills.org/2025/11/with-a-heavy-heart...
2,2025-11-10-02-00-00 +0000,wsj,Feeling Great About the Economy? You Must Own ...,https://www.wsj.com/economy/consumers/economy-...
3,2025-11-10-02-00-00 +0000,wsj,Paper wealth is powering this economy. And it ...,https://www.wsj.com/economy/consumers/economy-...
4,2025-11-10-01-55-10 +0000,48hills,Rep. Nancy Pelosi’s real legacy,https://48hills.org/2025/11/rep-nancy-pelosis-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
69,trump,31
50,shutdown,17
143,new,13
213,nyc,9
288,bbc,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
108,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,81
115,2025-11-09-18-12-48 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,73
113,2025-11-09-18-33-00 +0000,nypost,BBC head Tim Davie resigns over doctored foota...,https://nypost.com/2025/11/09/world-news/bbc-h...,65
101,2025-11-09-19-36-13 +0000,cbc,Trump administration wants U.S. states to 'und...,https://www.cbc.ca/news/world/trump-administra...,62
107,2025-11-09-18-48-33 +0000,cbc,"BBC director, head of news resign after critic...",https://www.cbc.ca/news/world/bbc-director-res...,62


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
108,81,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
113,34,2025-11-09-18-33-00 +0000,nypost,BBC head Tim Davie resigns over doctored foota...,https://nypost.com/2025/11/09/world-news/bbc-h...
93,32,2025-11-09-20-04-25 +0000,nypost,Flyers face fresh travel hell as government sh...,https://nypost.com/2025/11/09/us-news/flyers-f...
178,32,2025-11-09-09-00-00 +0000,cbc,NDP MP ‘very confident’ no New Democrats will ...,https://www.cbc.ca/news/politics/ndp-rejects-f...
31,31,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
163,31,2025-11-09-10-26-24 +0000,cbc,Super Typhoon Fung-wong hits Philippines after...,https://www.cbc.ca/news/world/super-typhoon-th...
8,26,2025-11-10-01-24-00 +0000,wsj,The Senate was driving toward a deal to end th...,https://www.wsj.com/politics/policy/republican...
160,26,2025-11-09-10-59-00 +0000,startribune,Taylor Schlitz: Would you go on a date for fre...,https://www.startribune.com/young-people-datin...
117,26,2025-11-09-17-30-00 +0000,wsj,President Trump suggests tariff revenue could ...,https://www.wsj.com/politics/policy/trump-sugg...
124,23,2025-11-09-15-56-32 +0000,nypost,Transportation secretary gives dire warning on...,https://nypost.com/2025/11/09/us-news/transpor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
